In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mido
import numpy as np
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Flatten
from tensorflow.keras.models import Model

In [ ]:
import glob
import mido
from mido import MidiFile, MidiTrack, Message
count = 0
off = 0
mid = MidiFile('/content/drive/MyDrive/Major Project/datasets/classical_midi/mozart/mz_331_2.mid')
#print(midi)
for msg in mid:
  if not msg.is_meta and msg.type == 'note_on':
    if msg.velocity == 0:
      off += 1
    else:
      count += 1

  if not msg.is_meta and msg.type == "note_off":
    off += 1

In [ ]:
count

2681

In [ ]:
import glob
import mido
from mido import MidiFile, MidiTrack, Message
notes = []
times = []
to_delete = []
count = 0
for folder in glob.glob('/content/drive/MyDrive/Major Project/datasets/classical_midi/*'):
  for midi in glob.glob(folder + '/*.mid'):
    #print(midi)
    print(count)
    try:
      mid = MidiFile(midi)
      for msg in mid:
        if not msg.is_meta and msg.type == 'note_on':
          notes.append(msg.note)
          times.append(msg.time)
              
    except:
      to_delete.append(midi)
      print("error " + str(count))

    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
len(notes)

1521514

In [ ]:
np.save("/content/drive/MyDrive/Major Project/datasets/classical_notes.npy", np.array(notes))
np.save("/content/drive/MyDrive/Major Project/datasets/classical_times.npy", np.array(times))

In [ ]:
notes = np.load("/content/drive/MyDrive/Major Project/datasets/classical_notes.npy")
times = np.load("/content/drive/MyDrive/Major Project/datasets/classical_times.npy")

In [ ]:
pitchnames = sorted(set(item for item in notes))
print(pitchnames)
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print(note_to_int)

[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107]
{21: 0, 22: 1, 23: 2, 24: 3, 25: 4, 26: 5, 27: 6, 28: 7, 29: 8, 30: 9, 31: 10, 32: 11, 33: 12, 34: 13, 35: 14, 36: 15, 37: 16, 38: 17, 39: 18, 40: 19, 41: 20, 42: 21, 43: 22, 44: 23, 45: 24, 46: 25, 47: 26, 48: 27, 49: 28, 50: 29, 51: 30, 52: 31, 53: 32, 54: 33, 55: 34, 56: 35, 57: 36, 58: 37, 59: 38, 60: 39, 61: 40, 62: 41, 63: 42, 64: 43, 65: 44, 66: 45, 67: 46, 68: 47, 69: 48, 70: 49, 71: 50, 72: 51, 73: 52, 74: 53, 75: 54, 76: 55, 77: 56, 78: 57, 79: 58, 80: 59, 81: 60, 82: 61, 83: 62, 84: 63, 85: 64, 86: 65, 87: 66, 88: 67, 89: 68, 90: 69, 91: 70, 92: 71, 93: 72, 94: 73, 95: 74, 96: 75, 97: 76, 98: 77, 99: 78, 100: 79, 101: 80, 10

In [ ]:
from sklearn.preprocessing import MinMaxScaler
notes_scaler = MinMaxScaler(feature_range=(0,1))
notes_scaler.fit(np.array(notes).reshape(-1,1))
scaled_notes = list(notes_scaler.transform(np.array(notes).reshape(-1,1)))
times_scaler = MinMaxScaler(feature_range=(0.1,1))
times_scaler.fit(np.array(times).reshape(-1,1))
scaled_times = list(times_scaler.transform(np.array(times).reshape(-1,1)))

In [ ]:
sc_times = [np.log(time) for time in times]
sc_times[29]
times[29]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


0.042553166666666656

In [ ]:
notes_with_times = []
i = 0
for note in list(scaled_notes):
    temp = []
    temp.append(note)
    temp.append(list(scaled_times[i]))
    notes_with_times.append(temp)
    i += 1
notes_with_times = np.array(notes_with_times).reshape(-1, 2)
notes_with_times.shape

(1521514, 2)

In [ ]:
notes_with_times = [list(note) for note in notes_with_times]

X = []

# number of notes in a sequence
n_prev = 100

#for i in range(0, len(notes_with_times)-n_prev, 10):
  #X.append(notes_with_times[i : i + n_prev])
  
for i in range(0, len(notes_with_times) - n_prev, n_prev):
  X.append(notes_with_times[i : i + n_prev])

X = np.array(X)
print(X.shape)

(15215, 100, 2)


In [ ]:
X = np.array(X)
print(X[0,22,:])
print(X.shape)
np.random.shuffle(X)
print(X[0,22,:])
print(X.shape)

[0.53488372 0.1       ]
(15215, 100, 2)
[0.31395349 0.1       ]
(15215, 100, 2)


In [ ]:
def generator():
    gen_input = Input(shape = (None, 100))
    X = LSTM(256, return_sequences = True)(gen_input)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = LSTM(64, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    out = Dense(2, activation = "relu")(X)
    model = Model(inputs = gen_input, outputs = out)
    return model

In [ ]:
def discriminator():
    disc_input = Input(shape = (100, 2))
    X = LSTM(128, return_sequences = True)(disc_input)
    X = Dropout(0.3)(X)
    X = LSTM(64, return_sequences = True)(X)
    X = Dropout(0.3)(X)
    X = Flatten()(X)
    X = Dense(128, activation = "relu")(X)
    X = Dropout(0.3)(X)
    out = Dense(1, activation = "sigmoid")(X)
    model = Model(inputs = disc_input, outputs = out)
    return model

In [ ]:
def disc_with_gen(generator, discriminator):
    inputs = Input(shape = (None, 100))
    generated = generator(inputs)
    outputs = discriminator(generated)
    model = Model(inputs = inputs, outputs = outputs)
    return model

In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(0.0002, 0.5)
d = discriminator()
d.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
g = generator()

In [ ]:
d.trainable = False
combined = disc_with_gen(g, d)
combined.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
X.shape

(15215, 100, 2)

In [ ]:
from matplotlib import pyplot as plt
assert not np.any(np.isnan(X))

In [ ]:
disc_loss = []
disc_loss_epoch = []
gen_loss = []
gen_loss_epoch = []
batch_size = 128

In [ ]:
idx = list(range(0, batch_size, 1))
real_data = X[idx]
real_data.shape

(128, 100, 2)

In [ ]:
def train(epochs, num_of_batches, batch_size = batch_size):
    for epoch in range(epochs):
        a = 0
        b = batch_size
        for num in range(num_of_batches):
          idx = list(range(a, b, 1))
          real_data = X[idx]

          noise = np.random.normal(0, 1, (batch_size, 100, 100))

          generated = g.predict(noise)
          d_loss_real = d.train_on_batch(real_data, np.ones((batch_size, 1)))
          #print("d_loss_real: ", d_loss_real)
          d_loss_fake = d.train_on_batch(generated, np.zeros((batch_size, 1)))
          #print("d_loss_fake: ",d_loss_fake)
          d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
          print("%d d_loss: " %epoch,d_loss)
            
          noise = np.random.normal(0, 1, (batch_size, 100, 100))
          g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
          print("%d g_loss: " %epoch, g_loss)

          print("\n")

          disc_loss.append(d_loss[0])
          gen_loss.append(g_loss[0])

          a = a + batch_size
          b = b + batch_size
            
        if epoch % 1 == 0:
          print ("Epoch %d [D loss: %f, acc.: %.2f%%] [G loss: %f, acc.: %.2f%%]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], 100*g_loss[1]))
          print("\n")
          disc_loss_epoch.append(d_loss[0])
          gen_loss_epoch.append(g_loss[0])
          checkpoint = tf.train.Checkpoint(g)
          checkpoint.save('/content/drive/MyDrive/Major Project/checkpoints/training_checkpoints_classical')
            
    plt.plot(disc_loss, c='red')
    plt.plot(gen_loss, c='blue')
    plt.title("GAN Loss per batch")
    plt.legend(['Discriminator', 'Generator'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig('/content/drive/MyDrive/Major Project/output/GAN_Loss_per_Batch_classical.png', transparent=False)
    plt.close()

    plt.plot(disc_loss_epoch, c='red')
    plt.plot(gen_loss_epoch, c='blue')
    plt.title("GAN Loss per Epoch")
    plt.legend(['Discriminator', 'Generator'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.savefig('/content/drive/MyDrive/Major Project/output/GAN_Loss_per_Epoch_classical.png', transparent=False)
    plt.close()

In [ ]:
print(real_data.shape)
generated = g.predict(np.random.normal(0, 1, (batch_size, 100, 100)))
print(generated.shape)
#idx = list(range(0, batch_size, 1))
#print(real_data[idx])

(128, 100, 2)
(128, 100, 2)


In [ ]:
train(epochs = 40, num_of_batches = int(X.shape[0]/batch_size))

Streaming output truncated to the last 5000 lines.
29 g_loss:  [0.695181131362915, 0.0]


29 d_loss:  [0.69316596 0.5       ]
29 g_loss:  [0.6951807737350464, 0.0]


29 d_loss:  [0.69316599 0.5       ]
29 g_loss:  [0.6951802968978882, 0.0]


29 d_loss:  [0.69316593 0.5       ]
29 g_loss:  [0.6951799392700195, 0.0]


29 d_loss:  [0.69316596 0.5       ]
29 g_loss:  [0.6951795816421509, 0.0]


29 d_loss:  [0.69316599 0.5       ]
29 g_loss:  [0.6951792240142822, 0.0]


29 d_loss:  [0.69316599 0.5       ]
29 g_loss:  [0.695178747177124, 0.0]


29 d_loss:  [0.69316596 0.5       ]
29 g_loss:  [0.6951783895492554, 0.0]


29 d_loss:  [0.69316599 0.5       ]
29 g_loss:  [0.6951779723167419, 0.0]


29 d_loss:  [0.69316599 0.5       ]
29 g_loss:  [0.6951776146888733, 0.0]


29 d_loss:  [0.69316599 0.5       ]
29 g_loss:  [0.6951771378517151, 0.0]


29 d_loss:  [0.69316596 0.5       ]
29 g_loss:  [0.6951767802238464, 0.0]


29 d_loss:  [0.69316599 0.5       ]
29 g_loss:  [0.695176362991333, 0.0]




In [ ]:
g.save_weights("/content/drive/MyDrive/Major Project/saved_weights/classical/gen_model_weights_classical_120.h5")
d.save_weights("/content/drive/MyDrive/Major Project/saved_weights/classical/disc_model_weights_classical_120.h5")
combined.save_weights("/content/drive/MyDrive/Major Project/saved_weights/classical/combined_model_weights_classical_120.h5")

In [ ]:
g.load_weights("/content/drive/MyDrive/Major Project/saved_weights/Model_weights.h5")

In [ ]:
def save_models():
  d.trainable = False
  combined.save("/content/drive/MyDrive/Major Project/saved_models/classical/combined_model_classical_120.h5")
  d.trainable = True
  g.save("/content/drive/MyDrive/Major Project/saved_models/classical/gen_model_classical_120.h5")
  d.save("/content/drive/MyDrive/Major Project/saved_models/classical/disc_model_classical_120.h5")

In [ ]:
save_models()

In [ ]:
def save_loss_values():
  np.save("/content/drive/MyDrive/Major Project/saved_models/classical/disc_loss_120.npy", disc_loss)
  np.save("/content/drive/MyDrive/Major Project/saved_models/classical/gen_loss_120.npy", gen_loss)
  np.save("/content/drive/MyDrive/Major Project/saved_models/classical/disc_loss_epoch_120.npy", disc_loss_epoch)
  np.save("/content/drive/MyDrive/Major Project/saved_models/classical/gen_loss_epoch_120.npy", gen_loss_epoch)

In [ ]:
save_loss_values()

In [ ]:
g = tf.keras.models.load_model("/content/drive/MyDrive/Major Project/saved_models/gen_model_classical_40_epochs.h5")

In [ ]:
combined = tf.keras.models.load_model("/content/drive/MyDrive/Major Project/saved_models/combined_model_classical.h5")

In [ ]:
d.metrics_names[1]

'accuracy'

In [ ]:
pred = g.predict(np.random.normal(0, 1, (1, 100, 100)))
d.predict(pred)

array([[0.483567]], dtype=float32)

In [ ]:
d.predict(X[100, :, :].reshape(1, 100, 2))

array([[0.47452116]], dtype=float32)

In [ ]:
prediction = g.predict(np.random.normal(0, 1, (1, 100, 100)))
predicted_notes = []
predicted_times = []
for p in prediction.reshape(-1, 2):
    predicted_notes.append(p[0])
    predicted_times.append(p[1])
print(np.array(predicted_notes).shape)
print(np.array(predicted_times).shape)

(100,)
(100,)


In [ ]:
_notes = np.squeeze(notes_scaler.inverse_transform(np.array(predicted_notes).reshape(-1,1)))
final_notes = [int(i) for i in _notes]
print(final_notes)

[54, 71, 70, 62, 50, 47, 54, 60, 56, 62, 66, 60, 58, 61, 66, 68, 68, 61, 58, 52, 53, 52, 57, 57, 58, 59, 63, 60, 55, 59, 59, 57, 60, 61, 63, 61, 59, 55, 57, 56, 59, 64, 66, 67, 63, 58, 57, 57, 57, 65, 68, 66, 62, 64, 63, 61, 59, 58, 60, 61, 59, 58, 57, 56, 55, 55, 61, 63, 65, 64, 61, 58, 58, 58, 56, 59, 60, 63, 63, 65, 62, 63, 61, 58, 54, 54, 62, 69, 68, 63, 60, 58, 56, 54, 54, 61, 69, 69, 64, 60]


In [ ]:
predicted_times

[0.09137883,
 0.113375604,
 0.11239231,
 0.10335362,
 0.09119511,
 0.088635914,
 0.09717902,
 0.10196029,
 0.09898217,
 0.106539115,
 0.111029446,
 0.1048813,
 0.104096144,
 0.106125906,
 0.10970703,
 0.110072896,
 0.10873875,
 0.10302131,
 0.10188627,
 0.098399565,
 0.10000941,
 0.098289475,
 0.10147005,
 0.09852849,
 0.100519836,
 0.10062286,
 0.10344273,
 0.09887925,
 0.09450798,
 0.099985,
 0.09848659,
 0.095635146,
 0.09959483,
 0.09766053,
 0.09965514,
 0.09859066,
 0.09744263,
 0.09526928,
 0.09904046,
 0.09829892,
 0.09992786,
 0.102864,
 0.10296127,
 0.10378973,
 0.099016875,
 0.09432185,
 0.09611199,
 0.09684019,
 0.09686929,
 0.104220346,
 0.10655239,
 0.103920445,
 0.10245286,
 0.10728836,
 0.10770713,
 0.10662626,
 0.10522933,
 0.10344759,
 0.10407306,
 0.10319726,
 0.100146934,
 0.10033946,
 0.1000327,
 0.09989846,
 0.10126294,
 0.10096063,
 0.10451712,
 0.103177205,
 0.10254276,
 0.10183325,
 0.09941362,
 0.09642804,
 0.09802024,
 0.096454814,
 0.09335749,
 0.09746523,
 

In [ ]:
times_scaler = MinMaxScaler(feature_range=(0.08,1))
times_scaler.fit(np.array(times).reshape(-1,1))
final_times = np.squeeze(times_scaler.inverse_transform(np.array(predicted_times).reshape(-1,1)))
for i in range(100):
    if final_times[i] < 0:
        final_times[i] = 0
print(final_times)

[0.14479953 0.42471606 0.41220325 0.29718283 0.14246157 0.10989497
 0.2186089  0.27945217 0.24155463 0.33771935 0.39486036 0.31662312
 0.3066317  0.33246112 0.37803212 0.38268793 0.36571047 0.29295406
 0.27851033 0.23414078 0.25462663 0.23273985 0.2732138  0.23578139
 0.26112196 0.26243302 0.29831678 0.24024491 0.184619   0.25431603
 0.23524818 0.1989626  0.24935097 0.22473627 0.25011837 0.23657249
 0.22196342 0.1943068  0.24229644 0.23286007 0.25358883 0.2909522
 0.29219007 0.3027325  0.24199626 0.18225043 0.2050306  0.21429718
 0.21466751 0.30821222 0.3378883  0.30439588 0.28572035 0.34725377
 0.35258272 0.33882827 0.32105193 0.2983786  0.30633798 0.29519314
 0.25637665 0.25882658 0.25492302 0.25321472 0.27057824 0.26673117
 0.31198874 0.29493788 0.28686434 0.27783555 0.24704497 0.20905241
 0.22931375 0.20939316 0.16997863 0.22225107 0.22412929 0.23609711
 0.24706677 0.2918607  0.25397453 0.31058818 0.29653963 0.27138186
 0.25360495 0.24832548 0.32400018 0.37686956 0.3349429  0.28044

In [ ]:
from mido import MidiFile, MidiTrack, Message
mid = MidiFile()
track = MidiTrack()
for i in range(100):
    # 147 means note_on
    # 67 is velocity
    if final_notes[i] > 127:
      final_notes[i] = 127
    note = np.asarray([147, final_notes[i], 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    time = final_times[i]
    msg.time = round(time * mid.ticks_per_beat * 2)
    track.append(msg)
mid.tracks.append(track)
mid.save('/content/drive/MyDrive/Major Project/output/LSTM_music_time_classical.mid')

In [ ]:
from mido import MidiFile, MidiTrack, Message
mid = MidiFile()
track = MidiTrack()
t = 230
track.append(Message('program_change', program = 0, channel = 0, time = 0))
for note in final_notes:
    # 147 means note_on
    # 67 is velocity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    if t >= 160:
        t -= 2
    msg.time = int(t)
    msg.channel = 0
    track.append(msg)
mid.tracks.append(track)
mid.save('/content/drive/MyDrive/Major Project/output/LSTM_music_classical_check.mid')
for msg in mid:
  print(msg)

program_change channel=0 program=0 time=0
note_on channel=0 note=70 velocity=67 time=0.2375
note_on channel=0 note=71 velocity=67 time=0.23541666666666666
note_on channel=0 note=52 velocity=67 time=0.23333333333333334
note_on channel=0 note=48 velocity=67 time=0.23124999999999998
note_on channel=0 note=49 velocity=67 time=0.22916666666666666
note_on channel=0 note=50 velocity=67 time=0.22708333333333333
note_on channel=0 note=47 velocity=67 time=0.225
note_on channel=0 note=54 velocity=67 time=0.22291666666666665
note_on channel=0 note=59 velocity=67 time=0.22083333333333333
note_on channel=0 note=64 velocity=67 time=0.21875
note_on channel=0 note=64 velocity=67 time=0.21666666666666667
note_on channel=0 note=61 velocity=67 time=0.21458333333333332
note_on channel=0 note=58 velocity=67 time=0.2125
note_on channel=0 note=56 velocity=67 time=0.21041666666666667
note_on channel=0 note=55 velocity=67 time=0.20833333333333334
note_on channel=0 note=54 velocity=67 time=0.20625
note_on channe

In [ ]:
import mido
from mido import MidiFile, MidiTrack, Message
import numpy as np

#mid = MidiFile('/content/drive/MyDrive/Major Project/datasets/maestro/maestro-v3.0.0-2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi')
mid = MidiFile('/content/drive/MyDrive/Major Project/pnowks/corner.mid') 
notes = []

for msg in mid:
  print(msg)

print(mid.length)

Streaming output truncated to the last 5000 lines.
note_on channel=0 note=61 velocity=83 time=0.029166666666666667
note_on channel=0 note=58 velocity=72 time=0
control_change channel=0 control=64 value=0 time=0
note_on channel=0 note=73 velocity=75 time=0.029166666666666667
note_on channel=0 note=70 velocity=74 time=0
control_change channel=0 control=64 value=127 time=0.058333333333333334
note_on channel=0 note=70 velocity=0 time=0.0625
note_on channel=0 note=73 velocity=0 time=0.0375
note_on channel=0 note=61 velocity=0 time=0.03333333333333333
note_on channel=0 note=65 velocity=80 time=0
note_on channel=0 note=58 velocity=0 time=0.0125
note_on channel=0 note=65 velocity=0 time=0.0625
note_on channel=0 note=63 velocity=72 time=0.058333333333333334
note_on channel=0 note=63 velocity=0 time=0.03333333333333333
note_on channel=0 note=63 velocity=71 time=0.08333333333333333
note_on channel=0 note=63 velocity=0 time=0.0625
note_on channel=0 note=63 velocity=67 time=0.0625
note_on channel=0

In [ ]:
from mido import MidiFile

mid = MidiFile('/content/drive/MyDrive/Major Project/pnowks/corner.mid')
mididict = []
output = []

# Put all note on/off in midinote as dictionary.
for i in mid:
    if i.type == 'note_on' or i.type == 'note_off' or i.type == 'time_signature':
        mididict.append(i.dict())
# change time values from delta to relative time.
mem1=0
for i in mididict:
    time = i['time'] + mem1
    i['time'] = time
    mem1 = i['time']
# make every note_on with 0 velocity note_off
    if i['type'] == 'note_on' and i['velocity'] == 0:
        i['type'] = 'note_off'
# put note, starttime, stoptime, as nested list in a list. # format is [type, note, time, channel]
    mem2=[]
    if i['type'] == 'note_on' or i['type'] == 'note_off':
        mem2.append(i['type'])
        mem2.append(i['note'])
        mem2.append(i['time'])
        mem2.append(i['channel'])
        output.append(mem2)
# put timesignatures
    if i['type'] == 'time_signature':
        mem2.append(i['type'])
        mem2.append(i['numerator'])
        mem2.append(i['denominator'])
        mem2.append(i['time'])
        output.append(mem2)
# viewing the midimessages.
for i in output:
    print(i)
print(mid.ticks_per_beat)

Streaming output truncated to the last 5000 lines.
['note_off', 60, 506.69999999998936, 0]
['note_on', 66, 506.7458333333227, 0]
['note_on', 62, 506.87916666665603, 0]
['note_off', 66, 506.891666666656, 0]
['note_off', 72, 506.92083333332266, 0]
['note_on', 68, 506.954166666656, 0]
['note_off', 62, 506.96249999998935, 0]
['note_off', 68, 507.06249999998937, 0]
['note_on', 60, 507.11666666665604, 0]
['note_on', 72, 507.12499999998937, 0]
['note_off', 60, 507.1624999999894, 0]
['note_on', 66, 507.28333333332273, 0]
['note_off', 48, 507.3291666666561, 0]
['note_on', 62, 507.3874999999894, 0]
['note_off', 72, 507.4499999999894, 0]
['note_off', 66, 507.47916666665606, 0]
['note_off', 62, 507.4833333333227, 0]
['note_on', 68, 507.4833333333227, 0]
['note_off', 68, 507.63749999998936, 0]
['note_on', 50, 507.6583333333227, 0]
['note_on', 58, 507.66249999998934, 0]
['note_on', 74, 507.716666666656, 0]
['note_on', 65, 507.82499999998936, 0]
['note_off', 58, 507.9333333333227, 0]
['note_off', 65,

In [ ]:
note_duration = []
for o in enumerate(output):
  if o[0] == "note_off":
    note = o[1]
    stop_time = o[2]
    for idx, o in enumerate(output):
      if o[0] == "note_on" and o[1] == note:
        start_time = o[2]
        duration = stop_time - start_time
        temp = []
        temp.append(note)
        temp.append(duration)
        note_duration.append(temp)
        print(note_duration)
        break
    break

In [ ]:
print(output[0:10])

[['time_signature', 4, 4, 0], ['note_on', 36, 0.13333333333333333, 0], ['note_on', 48, 0.38749999999999996, 0], ['note_on', 55, 0.8999999999999999, 0], ['note_on', 60, 1.0208333333333333, 0], ['note_off', 55, 1.0333333333333332, 0], ['note_off', 60, 1.1583333333333332, 0], ['note_on', 62, 1.1749999999999998, 0], ['note_off', 62, 1.320833333333333, 0], ['note_on', 64, 1.345833333333333, 0]]
